## Petrinex Data Pipeline
### 2021-04-17 Colin Bieberstein

### Download Alberta Public Data - (Infrastrure Data) from https://www.petrinex.ca/PD/Pages/APD.aspx 


In [19]:
import time
import os as os
import shutil as shu
from bs4 import BeautifulSoup
from requests_html import HTMLSession
from urllib.parse import urljoin
import zipfile
import pyarrow.csv as pv
import pyarrow.parquet as pq
import pandas as pd
from icecream import ic


In [20]:
# Pandas Notebook formatting
pd.set_option('display.max_rows', 255)
pd.set_option('display.max_columns', 30)

# Matplotlib Notebook Inline diagrams
%matplotlib inline


### 0. Get ZIP file from Petrinex website with all the Infrastructure + Volumetric data included.

In [23]:
! python scrape.py

# NOTE You can't use this code inside a notebook so its in scrape.py for convienence.

# petrinex_pubdata = "https://www.petrinex.gov.ab.ca/PublicData"
# def get_all_forms(url):
#     """Returns all form tags found on a web page's `url` """
#     # GET request
#     res = session.get(url)
#     # for javascript driven website -> res.html.html will contain the html
#     # after javascript execution (render)
#     res.html.render()
#     soup = BeautifulSoup(res.html.html, "html.parser")
#     return soup.find_all("form")


# def get_form_details(form):
#     """Returns the HTML details of a form,
#     including action, method and list of form controls (inputs, etc)"""
#     details = {}
#     # get the form action (requested URL)
#     action = form.attrs.get("action").lower()
#     # get the form method (POST, GET, DELETE, etc)
#     # if not specified, GET is the default in HTML
#     method = form.attrs.get("method", "get").lower()
#     # get all form inputs
#     inputs = []
#     for input_tag in form.find_all("input"):
#         # get type of input form control
#         input_type = input_tag.attrs.get("type", "text")
#         # get name attribute
#         input_name = input_tag.attrs.get("name")
#         # get the default value of that input tag
#         input_value = input_tag.attrs.get("value", "")
#         # add everything to that list
#         inputs.append({"type": input_type, "name": input_name, "value": input_value})
#     # put everything to the resulting dictionary
#     details["action"] = action
#     details["method"] = method
#     details["inputs"] = inputs
#     return details


# def download_zip_file(s, url, params):
#     local_filename = url.split("/")[-1]
#     local_filename += ".zip"

#     # NOTE the stream=True parameter below
#     with s.get(url, stream=True, params=params) as r:
#         r.raise_for_status()
#         with open(local_filename, "wb") as f:
#             for chunk in r.iter_content(chunk_size=8192):
#                 # If you have chunk encoded response uncomment if
#                 # and set chunk_size parameter to None.
#                 # if chunk:
#                 f.write(chunk)
#     return local_filename


# # initialize an HTTP session
# session = HTMLSession()
# res = session.get(petrinex_pubdata)
# res.html.render()

# # Get the first form from the site
# form = get_all_forms(petrinex_pubdata)[0]
# form_details = get_form_details(form)

# # Prepare data to submit
# data = ""
# data = {}
# for input_tag in form_details["inputs"]:
#     nm = input_tag["name"]
#     va = input_tag["value"]
#     ty = input_tag["type"]
#     # Apply our custom logic for the Petrinex form
#     if ty == "hidden":
#         # if it's hidden, and not one of these use the default value
#         if "Download.DownloadCountry" in nm:
#             data[nm] = "Canada"
#         elif "Download.DownloadIP" in nm:
#             #
#             # Need to make this YOUR external IP.
#             #
#             data[nm] = "8.8.8.8"  
#             #
#         elif "Download.VolumetricDateFrom" in nm:
#             data[nm] = "2017-01-01"
#         elif "IsChecked" in nm:
#             pass
#             # Note the Petrinex site submits each of the checkbox params twice
#             # once as true (if checked) and again as false (hidden) no freaking
#             # clue whats up with that.  We're just skipping those hidden fields
#             # to see if it works
#         else:
#             data[nm] = va
#     elif ty != "submit":
#         if nm is not None:
#             if "IsChecked" in nm:
#                 data[nm] = "true"
#             elif "FileFormat" in nm:
#                 data[nm] = "csv"
# time.sleep(0.1)


# # Get the details that the site requires. (On the page it calls a seperate API via jquery)
# # We work around by hardcoding our external IP and matching location data. Then submitting 
# # the request to download a zip file (it returns a guid used to actually download.)

# url = urljoin(petrinex_pubdata, "/PublicData/Files/RequestZipFiles")
# session.headers.update(
#     {
#         "Accept": "content-type: application/json; charset=utf-8",
#         "Accept-Encoding": "gzip, deflate, br",
#         "Accept-Language": "en-US,en;q=0.5",
#         "Connection": "keep-alive",
#         "Content-Type": "application/x-www-form-urlencoded; charset=UTF-8",
#         "Origin": "https://www.petrinex.gov.ab.ca",
#         "referrer": "https://www.petrinex.gov.ab.ca/PublicData",
#         "X-Requested-With": "XMLHttpRequest",
#     }
# )
# res2 = session.post(url, data=data)
# time.sleep(0.1)
# json = res2.json()
# time.sleep(0.1)

# # Use the json data from the last request to build a payload for this request.
# data = {"arg_strGUID": json["DownloadGUID"]}
# url = urljoin(petrinex_pubdata, "/PublicData/Files/DownloadFile")
# print("Downloading, takes 8-10 minutes...")
# filename = download_zip_file(session, url, data)
# print(filename)


DownloadFile.zip


### 1. Extract from ZIP file in current dir to ./temp/

In [24]:
with zipfile.ZipFile("./DownloadFile.zip", 'r') as zip_ref:
    zip_ref.extractall("./temp/")

### 2. Process the extracted .csv.zip files into parquet files

In [25]:
with os.scandir("./temp/") as it:
    for entry in it:
        if entry.name.endswith(".csv.zip") and entry.is_file():           
            # Read the zipped csv files
            ic(entry.name)
            table = pd.read_csv(entry.path, 
                                header=0, 
                                compression='zip', 
                                sep=',', 
                                quotechar='"', 
                                low_memory=False)
            
            # Save the file in parquet format using gzip compression
            table.to_parquet(entry.name.replace(".csv.",".parquet.").replace(".zip",'.gz'),
                             compression='gzip')
            
            # Explicitly free memory
            del(table)
            
# Cleanup the ./temp/ directory and files
shu.rmtree("./temp")
# Should delete the zip file


ic| entry.name: 'Business Associate.csv.zip'
ic| entry.name: 'Facility Infrastructure.csv.zip'
ic| entry.name: 'Facility Licence.csv.zip'
ic| entry.name: 'Facility Operator History.csv.zip'
ic| entry.name: 'Vol_2017-01.csv.zip'
ic| entry.name: 'Vol_2017-02.csv.zip'
ic| entry.name: 'Vol_2017-03.csv.zip'
ic| entry.name: 'Vol_2017-04.csv.zip'
ic| entry.name: 'Vol_2017-05.csv.zip'
ic| entry.name: 'Vol_2017-06.csv.zip'
ic| entry.name: 'Vol_2017-07.csv.zip'
ic| entry.name: 'Vol_2017-08.csv.zip'
ic| entry.name: 'Vol_2017-09.csv.zip'
ic| entry.name: 'Vol_2017-10.csv.zip'
ic| entry.name: 'Vol_2017-11.csv.zip'
ic| entry.name: 'Vol_2017-12.csv.zip'
ic| entry.name: 'Vol_2018-01.csv.zip'
ic| entry.name: 'Vol_2018-02.csv.zip'
ic| entry.name: 'Vol_2018-03.csv.zip'
ic| entry.name: 'Vol_2018-04.csv.zip'
ic| entry.name: 'Vol_2018-05.csv.zip'
ic| entry.name: 'Vol_2018-06.csv.zip'
ic| entry.name: 'Vol_2018-07.csv.zip'
ic| entry.name: 'Vol_2018-08.csv.zip'
ic| entry.name: 'Vol_2018-09.csv.zip'
ic| entry.na

### Zip up the results for fun

In [26]:
# specifying the zip file name
zip_name = "parquet_petrinex_data.zip"
  
# writing files to a zipfile
with zipfile.ZipFile(zip_name,'w') as zip:
    with os.scandir("./") as it:
        for entry in it:
            if entry.name.endswith(".parquet.gz") and entry.is_file():  
                # Some optimistic (shitty) thinking here zip + delete w/o checking either for success
                # aka non-production code.
                zip.write(entry.name)
                os.remove(entry.path)

# TO DO

* Add in code to change to categorical / etc column types before writing parquet files. Use a pattern like name contains ('Business Associates')... ('VOl_YYYY_NN_DD') etc.
* Add in even rudimentary error handling, right now it just fails and you use the notebook to troubleshoot
* Refactor the download code to something re-usable, right now it has some very specific logic for Petrinex
* Refactor the Download_zip_file code to just download a file, but to accept the name as a parameter instead of trusting the remote name (and appending .zip)
* Also note... the download code only works externally to a notebook because the notebook has an async loop going.   FIx that or just ! python scrape.py instead
